In [2]:
import os
import tarfile
import urllib
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import dask.dataframe as dd
import dask.array as da
import dask.bag as dbs
import warnings
warnings.filterwarnings('ignore')


In [3]:
ddf = dd.read_csv('Iowa_Liquor_Sales.csv', dtype = {'Category': 'float64',
                                                    'County Number': 'float64',
                                                    'Zip Code': 'object',
                                                    'Vendor Number': 'float64',
                                                    'Item Number': 'object',
                                                      'State Bottle Cost': 'object',
                                                       'State Bottle Retail': 'object'},
                                            blocksize = '10MB')

In [5]:
ddf.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')

In [30]:
#split main df into smaller subcategory df's
county_info = ['County Number', 'County']
vendor_info = ['Vendor Name', 'Vendor Number']
product_info = ['Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)']
price_info = ['Item Number', 'State Bottle Cost', 'State Bottle Retail', 'Date']
store_info = ['Store Number', 'Store Name', 'City', 'Zip Code', 'County Number', "County"]
transaction_info = ['Vendor Number','Invoice/Item Number', 'Date', 'Store Number', 
                    'Category', 'Item Number', 'Bottles Sold', 'Volume Sold (Gallons)',
                    'Sale (Dollars)']
category_info = ['Item Description','Category', 'Category Name']




# Counties Sub-Frame

In [31]:
#select county info columns
ddf_county = ddf[county_info]


In [32]:
#get rid of duplicate county rows
ddf_county = ddf_county.drop_duplicates().compute()
ddf_county

,County Number,County
0,14.0,CARROLL
1,97.0,WOODBURY
2,18.0,CHEROKEE
3,7.0,BLACK HAWK
4,16.0,CEDAR
...,...,...
2800,51.0,Jefferson
2887,20.0,Clarke
3067,36.0,Fremont
4677,46.0,Humboldt


In [60]:
#check for NA county numbers
ddf_county[ddf_county['County Number'].isna()]

,County Number,County
25143,NaN,NaN
39271,NaN,EL PASO


In [61]:
#drop rows w/ NA for county number
ddf_county = ddf_county[~ddf_county['County Number'].isna()]

#add in row for unknown
ddf_county = ddf_county.append(pd.Series({'County Number': 999, 'County': 'Unknown'}),
                               ignore_index = True)

ddf_county

,County Number,County
0,14.0,CARROLL
1,97.0,WOODBURY
2,18.0,CHEROKEE
3,7.0,BLACK HAWK
4,16.0,CEDAR
...,...,...
196,51.0,Jefferson
197,20.0,Clarke
198,36.0,Fremont
199,46.0,Humboldt


In [62]:
#gets rid of duplicates from typos, mispelling, etc
ddf_county = ddf_county.loc[ddf_county['County Number'].drop_duplicates().index]

#contains all  Iowa counties, plus row for unknown
ddf_county

,County Number,County
0,14.0,CARROLL
1,97.0,WOODBURY
2,18.0,CHEROKEE
3,7.0,BLACK HAWK
4,16.0,CEDAR
...,...,...
107,59.0,LUCAS
109,68.0,MONROE
110,2.0,ADAMS
113,93.0,WAYNE


In [63]:
#convert county numbers to integer type
ddf_county.loc[:, 'County Number'] = ddf_county['County Number'].astype(int)

#rename columns
ddf_county.columns = ['CountyNumber', 'County']

#reset the index
ddf_county = ddf_county.reset_index(drop = True)

#export df as csv
ddf_county.to_csv('county.csv')


# vendor

In [10]:
ddfv = ddf[vendor_info]
ddfv['Vendor Number'].nunique().compute()

411

In [66]:
ddfv = ddf[vendor_info]

#drop duplicate vendor rows
ddfv = ddfv.drop_duplicates().compute()

ddfv

,Vendor Name,Vendor Number
0,DIAGEO AMERICAS,260.0
1,BACARDI USA INC,35.0
2,LUXCO INC,434.0
3,MOET HENNESSY USA,420.0
5,Shaw-Ross International,460.0
...,...,...
1589,"Puente International, Inc.",656.0
33258,Chuckanut Bay Distillery,694.0
15508,Mexcor Inc.,408.0
16101,Baron Nahmias Inc,329.0


In [68]:
#rename columns
ddfv.columns = ['VendorName', 'VendorNumber']

#prepping to check for NA values in columns
idx_na = ddfv['VendorNumber'].isna()
name_na = ddfv['VendorName'].isna()



In [69]:
#checks for NA vendor number
ddfv[idx_na]

,VendorName,VendorNumber
796,NaN,NaN
35886,Reservoir Distillery,NaN


In [70]:
#checks for NA vendor name
ddfv[name_na]

,VendorName,VendorNumber
796,NaN,NaN


In [71]:
ddfv

,VendorName,VendorNumber
0,DIAGEO AMERICAS,260.0
1,BACARDI USA INC,35.0
2,LUXCO INC,434.0
3,MOET HENNESSY USA,420.0
5,Shaw-Ross International,460.0
...,...,...
1589,"Puente International, Inc.",656.0
33258,Chuckanut Bay Distillery,694.0
15508,Mexcor Inc.,408.0
16101,Baron Nahmias Inc,329.0


In [72]:
#removes NA vendor name row
ddfv = ddfv[~ddfv['VendorNumber'].isna()]
ddfv

,VendorName,VendorNumber
0,DIAGEO AMERICAS,260.0
1,BACARDI USA INC,35.0
2,LUXCO INC,434.0
3,MOET HENNESSY USA,420.0
5,Shaw-Ross International,460.0
...,...,...
1589,"Puente International, Inc.",656.0
33258,Chuckanut Bay Distillery,694.0
15508,Mexcor Inc.,408.0
16101,Baron Nahmias Inc,329.0


In [73]:
#checking for repeat names
ddfv['VendorName'].value_counts()

Casa 1921 LLC                   2
Levecke Corporation             2
Best Case Wines LLC             1
The Country Vintner             1
RJ Importers LLC                1
                               ..
Fetzer Vineyards                1
Better Brands Bev Co            1
MISSISSIPPI RIVER DISTIL        1
JINRO AMERICA INC               1
Bonavita Beverage Group, LLC    1
Name: VendorName, Length: 565, dtype: int64

In [74]:
#check for repeat vendor numbers
ddfv['VendorNumber'].value_counts()

391.0    4
214.0    4
192.0    4
803.0    4
311.0    3
        ..
215.0    1
557.0    1
80.0     1
333.0    1
631.0    1
Name: VendorNumber, Length: 411, dtype: int64

In [75]:
#removes duplicate vendor numbers
ddfv = ddfv.drop_duplicates(subset = ['VendorNumber'])
ddfv

,VendorName,VendorNumber
0,DIAGEO AMERICAS,260.0
1,BACARDI USA INC,35.0
2,LUXCO INC,434.0
3,MOET HENNESSY USA,420.0
5,Shaw-Ross International,460.0
...,...,...
1589,"Puente International, Inc.",656.0
33258,Chuckanut Bay Distillery,694.0
15508,Mexcor Inc.,408.0
16101,Baron Nahmias Inc,329.0


In [76]:
#checks that all value counts are now 1
ddfv['VendorNumber'].value_counts(dropna = False)

386.0    1
324.0    1
886.0    1
565.0    1
749.0    1
        ..
160.0    1
731.0    1
643.0    1
621.0    1
485.0    1
Name: VendorNumber, Length: 411, dtype: int64

In [77]:
#add in row for unknown
ddfv = ddfv.append(pd.Series({'VendorNumber': 9999, 'VendorName': 'Unknown'}), ignore_index = True)
ddfv

,VendorName,VendorNumber
0,DIAGEO AMERICAS,260.0
1,BACARDI USA INC,35.0
2,LUXCO INC,434.0
3,MOET HENNESSY USA,420.0
4,Shaw-Ross International,460.0
...,...,...
407,Chuckanut Bay Distillery,694.0
408,Mexcor Inc.,408.0
409,Baron Nahmias Inc,329.0
410,Bull Run Distillery LLC,699.0


In [79]:
#checks that vendor numbers are integers
ddfv['VendorNumber'].describe()


count     412.000000
mean      445.337379
std       516.973610
min        10.000000
25%       255.750000
50%       419.000000
75%       580.500000
max      9999.000000
Name: VendorNumber, dtype: float64

In [82]:
#export df as csv
ddfv.to_csv('vendor.csv')

# Price

In [38]:
ddfp = ddf[price_info]

In [42]:
ddfp['Item Number'].nunique().compute()

11249

In [149]:
#drop duplicate price rows
ddfp = ddfp.drop_duplicates().compute()


In [150]:
ddfp

,Item Number,State Bottle Cost,State Bottle Retail,Date
0,11296,16.59,24.89,07/26/2021
1,43125,8.26,12.39,07/27/2021
2,89387,7.83,11.75,07/26/2021
3,48099,6.99,10.49,07/28/2021
4,36301,4.13,6.20,07/26/2021
...,...,...,...,...
13566,88205,16.00,24.00,07/23/2021
13572,65426,14.10,21.15,07/26/2021
13614,77054,12.96,19.44,07/23/2021
13684,4044,13.50,20.25,07/27/2021


In [151]:
#rename columns
ddfp.columns = ['ItemNumber', 'StateBottleCost', 'StateBottleRetail', 'Date']

#removes letter from item number
ddfp = ddfp.replace('x904631', '904631')

In [152]:
ddfp

,ItemNumber,StateBottleCost,StateBottleRetail,Date
0,11296,16.59,24.89,07/26/2021
1,43125,8.26,12.39,07/27/2021
2,89387,7.83,11.75,07/26/2021
3,48099,6.99,10.49,07/28/2021
4,36301,4.13,6.20,07/26/2021
...,...,...,...,...
13566,88205,16.00,24.00,07/23/2021
13572,65426,14.10,21.15,07/26/2021
13614,77054,12.96,19.44,07/23/2021
13684,4044,13.50,20.25,07/27/2021


In [153]:
#convert item numbers to integer type
ddfp.loc[:, 'ItemNumber'] = ddfp['ItemNumber'].astype(int)

#convert date column to date type
ddfp['Date'] = pd.to_datetime(ddfp['Date'], infer_datetime_format=True)

ddfp

,ItemNumber,StateBottleCost,StateBottleRetail,Date
0,11296,16.59,24.89,2021-07-26
1,43125,8.26,12.39,2021-07-27
2,89387,7.83,11.75,2021-07-26
3,48099,6.99,10.49,2021-07-28
4,36301,4.13,6.20,2021-07-26
...,...,...,...,...
13566,88205,16.00,24.00,2021-07-23
13572,65426,14.10,21.15,2021-07-26
13614,77054,12.96,19.44,2021-07-23
13684,4044,13.50,20.25,2021-07-27


In [155]:
#sort by date
ddfp = ddfp.sort_values(by='Date')
ddfp

,ItemNumber,StateBottleCost,StateBottleRetail,Date
28424,34034,5.71,8.56,2012-01-03
35955,41604,6.22,9.33,2012-01-03
6121,28238,26.22,39.33,2012-01-03
25339,42031,4.55,6.82,2012-01-03
1845,73526,6.30,9.45,2012-01-03
...,...,...,...,...
35124,56942,9.53,14.30,2022-04-29
35128,29421,3.66,5.49,2022-04-29
35132,101296,49.41,74.12,2022-04-29
35070,72532,10.49,15.74,2022-04-29


In [157]:
#filter out data before 2018
# ddf = 
ddfp[ddfp['Date'] >= '2018-01-01']


,ItemNumber,StateBottleCost,StateBottleRetail,Date
37816,44456,6.83,10.25,2018-01-02
37857,86506,3.42,5.13,2018-01-02
37779,40151,4.84,7.26,2018-01-02
37608,49086,27.97,41.96,2018-01-02
37888,86834,10.00,15.00,2018-01-02
...,...,...,...,...
35124,56942,9.53,14.30,2022-04-29
35128,29421,3.66,5.49,2022-04-29
35132,101296,49.41,74.12,2022-04-29
35070,72532,10.49,15.74,2022-04-29


In [158]:
#prepping to check for NA values in columns
idx_na = ddfp['ItemNumber'].isna()
cost_na = ddfp['StateBottleCost'].isna()
ret_na = ddfp['StateBottleRetail'].isna()
date_na = ddfp['Date'].isna()


In [159]:
#checks for NA item number
ddfp[idx_na]


,ItemNumber,StateBottleCost,StateBottleRetail,Date


In [160]:
#checks for NA bottle cost
ddfp[cost_na]


,ItemNumber,StateBottleCost,StateBottleRetail,Date
1634,38180,NaN,NaN,2012-01-30
2284,38180,NaN,NaN,2012-01-31
11050,35920,NaN,NaN,2012-01-31


In [161]:
#checks for NA bottle retail
ddfp[ret_na]


,ItemNumber,StateBottleCost,StateBottleRetail,Date
1634,38180,NaN,NaN,2012-01-30
2284,38180,NaN,NaN,2012-01-31
11050,35920,NaN,NaN,2012-01-31


In [162]:
#checks for NA date
ddfp[date_na]

,ItemNumber,StateBottleCost,StateBottleRetail,Date


In [164]:
#drop the rows with NAs. Three missing rows won't affect much
ddfp = ddfp.drop([1634, 2284, 11050])


KeyError: '[1634 2284 11050] not found in axis'

In [167]:
#export df as csv
ddfp.to_csv('price.csv')


# Stores Sub-Frame


In [12]:
ddfs = ddf[store_info]

# ddfs['Store Number'].nunique().compute()

In [ ]:
ddfs

In [13]:
ddfs=ddfs.dropna(subset=["Zip Code"])
ddfs['Store Number'].nunique().compute()

2729

In [14]:
ddfs

,Store Number,Store Name,City,Zip Code,County Number,County
npartitions=580,,,,,,
,int64,object,object,object,float64,object
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [15]:
ddfs = ddfs.drop_duplicates().compute()
ddfs

,Store Number,Store Name,City,Zip Code,County Number,County
0,4001,Wal-Mart 1787 / Carroll,Carroll,51401,14.0,CARROLL
1,4568,Select Mart / Sioux City,Sioux City,51108,97.0,WOODBURY
2,5441,Liquor on the Corner,Cherokee,51012,18.0,CHEROKEE
3,5856,Station Mart #1 - Evansdale,Evansdale,50707,7.0,BLACK HAWK
4,5071,Jeff's Market / Durant,Durant,52747,16.0,CEDAR
...,...,...,...,...,...,...
5402,2190,Central City 2,Des Moines,50314,77.0,POLK
31603,6171,Speedee Mart 1515 / Council Bluffs,Des Moines,50310,77.0,POLK
19786,6172,BP to Go / Hiawatha,Hiawatha,52233,57.0,LINN
28596,5204,"MK Minimart, Inc",Burlington,52601,29.0,DES MOINES


In [17]:
ddfs = ddfs.drop_duplicates(subset = 'Store Number')

In [18]:
ddfs

,Store Number,Store Name,City,Zip Code,County Number,County
0,4001,Wal-Mart 1787 / Carroll,Carroll,51401,14.0,CARROLL
1,4568,Select Mart / Sioux City,Sioux City,51108,97.0,WOODBURY
2,5441,Liquor on the Corner,Cherokee,51012,18.0,CHEROKEE
3,5856,Station Mart #1 - Evansdale,Evansdale,50707,7.0,BLACK HAWK
4,5071,Jeff's Market / Durant,Durant,52747,16.0,CEDAR
...,...,...,...,...,...,...
28419,5867,k food mart / Monticello,Monticello,52310,53.0,JONES
13688,6088,New Starr / Fort Dodge,Fort Dodge,50501,94.0,WEBSTER
28779,9048,Artisan Grain Distillery,Davenport,52801,82.0,SCOTT
16060,6109,Moes Mart #11 / Sioux Center,Sioux Center,51250,84.0,SIOUX


In [20]:
#rename columns
ddfs.columns = ['StoreNumber', 'StoreName', 'City', 'ZipCode', 'CountyNumber', 'County']

#prepping to check for NA values in columns
idx_na = ddfs['StoreNumber'].isna()
name_na = ddfs['StoreName'].isna()
city_na = ddfs['City'].isna()
zip_na = ddfs['ZipCode'].isna()
county_na = ddfs['CountyNumber'].isna()


In [21]:
#checks for NA store number
ddfs[idx_na]

,StoreNumber,StoreName,City,ZipCode,CountyNumber,County


In [22]:
#checks for NA store name
ddfs[name_na]

,StoreNumber,StoreName,City,ZipCode,CountyNumber,County


In [23]:
#checks for NA city
ddfs[city_na]

,StoreNumber,StoreName,City,ZipCode,CountyNumber,County


In [24]:
#checks for NA zip code
ddfs[zip_na]

,StoreNumber,StoreName,City,ZipCode,CountyNumber,County


In [25]:
#checks for NA county number
ddfs[county_na]


,StoreNumber,StoreName,City,ZipCode,CountyNumber,County
12556,4222,STATION MART LIQUOR AND TOBACCO,EVANSDALE,50707,NaN,NaN
13292,3747,T-N-T Liquor,SIGOURNEY,52591,NaN,NaN
291,5249,W-Mart,Davenport,52803,NaN,NaN
27896,5284,Brooklyn Grocery,Brooklyn,52211,NaN,NaN
33606,5247,Rockwell Area Market,Rockwell,50469,NaN,NaN
34001,2460,"Liquor Barn, Inc.",Hampton,50441,NaN,NaN
34483,5278,Crossroads fo Spencer,Spencer,51301,NaN,NaN
3275,5224,Keystone Liquor & Wine / Coralville,CORALVILLE,52241,NaN,NaN
3509,5227,Flaming Hot Fish and Chicken LLC,CLINTON,52732,NaN,NaN
4206,4214,New Star Liquor / Lafayet / Waterloo,WATERLOO,50703,NaN,NaN


In [26]:
#convert store numbers to integer type
ddfs.loc[:, 'StoreNumber'] = ddfs['StoreNumber'].astype(int)


In [27]:
#export df as csv
ddfs.to_csv('store.csv')

# Transactions

In [181]:
ddft = ddf[transaction_info]

In [182]:
ddft = ddft.compute()

In [185]:
#rename columns
ddft.columns = ['InvoiceNumber', 'Date', 'StoreNumber', 'Category', 
               'VendorNumber', 'ItemNumber', 'BottlesSold', 'Volume(Gallons)','Sale']




In [199]:
#replace any commas in the bottlevolume column
ddft['BottlesSold']=ddft['BottlesSold'].str.replace(',','')
ddft['BottlesSold']=ddft['BottlesSold'].fillna(0)

In [200]:
#change invoice number to index
# ddft = ddft.set_index('InvoiceNumber')

#convert date column to date type
ddft['Date'] = pd.to_datetime(ddft['Date'], infer_datetime_format=True)

#convert store numbers to integer type
ddft.loc[:, 'StoreNumber'] = ddft['StoreNumber'].astype(int)

#convert category to integer type and fills na's as 9999999
ddft.loc[:, 'Category'] = ddft['Category'].fillna(9999999).astype(int)

#convert vendor numbers to integer type and fills na's as 9999
ddft.loc[:, 'VendorNumber'] = ddft['VendorNumber'].fillna(9999).astype(int)

#convert item numbers to integer type
ddft.loc[:, 'ItemNumber'] = ddft['ItemNumber'].astype(int)

#convert volume to integer type
#ddft.loc[:, 'Volume(Gallons)'] = ddft['Volume(Gallons)'].astype(int)

#convert bottles sold to integer type
ddft.loc[:, 'BottlesSold'] = ddft['BottlesSold'].astype(int)

#sort by date
ddft = ddft.sort_values(by='Date')

In [34]:
ddft.head(50)

NameError: name 'ddft' is not defined

In [206]:
#export df as csv
ddft.to_csv('transaction.csv')


# Categories


In [79]:
ddfc = ddf[category_info]
ddfc = ddfc.compute()

In [80]:
ddfc = ddfc.drop_duplicates('Category')
ddfc = ddfc.reset_index(drop = True)


In [81]:
ddfc

,Item Description,Category,Category Name
0,Crown Royal,1012100.0,Canadian Whiskies
1,Bacardi Superior PET,1062200.0,White Rum
2,Juarez Gold,1022100.0,Mixto Tequila
3,Hennessy VS,1052100.0,Imported Brandies
4,Hawkeye Vodka Mini,1031100.0,American Vodkas
...,...,...,...
108,Sam Adams Utopias HA,1501100.0,HIGH PROOF BEER - AMERICAN
109,Sobieski Orange,1032230.0,IMPORTED VODKA - CHERRY
110,Kraken Black Spiced Rum,1082300.0,SCHNAPPS - IMPORTED
111,Craigellachie,101220.0,NaN


In [82]:
ddfc[ddfc['Category Name'].isna()]


,Item Description,Category,Category Name
61,99 Watermelon Mini,NaN,NaN
105,Whaler's Original Dark Rum,1062250.0,NaN
111,Craigellachie,101220.0,NaN
112,Cocoyac Excellence,1089290.0,NaN


In [83]:
ddfc.at[105, 'Category Name'] = 'Rum'

ddfc.at[111, 'Category Name'] = 'whisky'

ddfc.at[112, 'Category Name'] = 'liquer'

In [84]:
ddfc[ddfc['Category Name'].isna()]


,Item Description,Category,Category Name
61,99 Watermelon Mini,NaN,NaN


In [90]:
ddfc = ddfc.drop(61)
ddfc.reset_index(drop = True)

,Item Description,Category,Category Name
0,Crown Royal,1012100.0,Canadian Whiskies
1,Bacardi Superior PET,1062200.0,White Rum
2,Juarez Gold,1022100.0,Mixto Tequila
3,Hennessy VS,1052100.0,Imported Brandies
4,Hawkeye Vodka Mini,1031100.0,American Vodkas
...,...,...,...
107,Sam Adams Utopias HA,1501100.0,HIGH PROOF BEER - AMERICAN
108,Sobieski Orange,1032230.0,IMPORTED VODKA - CHERRY
109,Kraken Black Spiced Rum,1082300.0,SCHNAPPS - IMPORTED
110,Craigellachie,101220.0,whisky


In [96]:
#drop item description
ddfc = ddfc.drop('Item Description', axis = 1)
#rename columns
ddfc.columns = ['CategoryNumber', 'CategoryName']
#convert category to integer type and fills na's as 9999999
ddfc.loc[:, 'Category'] = ddfc['Category'].fillna(9999999).astype(int)


In [100]:
#sort by category name
ddfc = ddfc.sort_values(by='CategoryName').reset_index(drop = True)

In [107]:
ddfc['Category'] = 'other'

In [134]:
a = ddfc.copy()

In [135]:
a.loc[a['CategoryName'].str.contains('VODKA|vodka|Vodka', regex = True), 'Category'] = 'Vodka'
a.loc[a['CategoryName'].str.contains('TEQUILA|tequila|Tequila', regex = True), 'Category'] = 'Tequila'
a.loc[a['CategoryName'].str.contains('AMARETTO|amaretto|Amaretto|ANISETTE|SCHNAPPS|Schnapps|Liqueur|liquer|Liqueurs|LIQEURS|LIQUEURS|CREME DE ALMOND|CREME DE CACAO|GREEN CREME|TRIPLE SEC|Triple Sec|CREME DE MENTHE', regex = True), 'Category'] = 'Liqueur'
a.loc[a['CategoryName'].str.contains('COCKTAILS|COCKTAIL|cocktails|cocktail|Cocktails|Cocktail', regex = True), 'Category'] = 'Cocktails'
a.loc[a['CategoryName'].str.contains('GINS|GIN|gins|gin|Gins|Gin', regex = True), 'Category'] = 'Gin'
a.loc[a['CategoryName'].str.contains('BRANDIES|BRANDY|brandies|brandy|Brandies|Brandy', regex = True), 'Category'] = 'Brandy'
a.loc[a['CategoryName'].str.contains('RUM|rum|Rum', regex = True), 'Category'] = 'Rum'
a.loc[a['CategoryName'].str.contains('WHISKIES|WHISKY|WHISKEY|whiskies|whisky|whiskey|Whiskies|Whisky|Whiskey|BOURBON|bourbon|Bourbon|SCOTCH|Scotch', regex = True), 'Category'] = 'Whiskey'
a.loc[a['CategoryName'].str.contains('ROCK & RYE', regex = False), 'Category'] = 'Liqueur'





In [136]:

na_row = {'CategoryNumber': 9999999, 'CategoryName': None, 'Category': None}
a = a.append(na_row, ignore_index = True)


a.columns = ['Category', 'CategoryName', 'SpecificCategory']


In [137]:
a

,Category,CategoryName,SpecificCategory
0,1022200.0,100% Agave Tequila,Tequila
1,1082010.0,AMARETTO - IMPORTED,Liqueur
2,1101100.0,AMERICAN ALCOHOL,other
3,1081010.0,AMERICAN AMARETTO,Liqueur
4,1051010.0,AMERICAN GRAPE BRANDIES,Brandy
...,...,...,...
108,1081600.0,Whiskey Liqueur,Whiskey
109,1062200.0,White Rum,Rum
110,1089290.0,liquer,Liqueur
111,101220.0,whisky,Whiskey


In [138]:
#export df as csv
a.to_csv('category.csv')

# Product info

In [120]:
#select product info columns
ddfproduct = ddf[product_info]

In [121]:
#drop duplicate product rows
ddfproduct = ddfproduct.drop_duplicates().compute()
ddfproduct

,Item Number,Item Description,Pack,Bottle Volume (ml)
0,11296,Crown Royal,12,750
1,43125,Bacardi Superior PET,12,750
2,89387,Juarez Gold,12,"1,000"
3,48099,Hennessy VS,24,200
4,36301,Hawkeye Vodka Mini,12,50
...,...,...,...,...
24301,966056,SOOH Nonino Quintessentia Amaro,6,750
25109,917079,Bull Run Distillery 15 year Bourbon,12,750
28183,944015,Rhum JM XO,6,750
28473,927274,Two Jays Light Whiskey,12,375


In [122]:
#rename columns
ddfproduct.columns = ['ItemNumber', 'ItemDescription', 'Pack', 'BottleVolume(ml)']

In [123]:
#prepping to check for NA values in columns
idx_na = ddfproduct['ItemNumber'].isna()
desc_na = ddfproduct['ItemDescription'].isna()
pack_na = ddfproduct['Pack'].isna()
vol_na = ddfproduct['BottleVolume(ml)'].isna()

In [124]:
#checks for NA item number
ddfproduct[idx_na]

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)


In [88]:
#checks for NA item description
ddfproduct[desc_na]

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)


In [89]:
#checks for NA pack
ddfproduct[pack_na]

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)


In [90]:
#checks for NA bottle volume
ddfproduct[vol_na]


,ItemNumber,ItemDescription,Pack,BottleVolume(ml)


In [125]:
#check for repeat item numbers
ddfproduct['ItemNumber'].value_counts()


965266    7
22219     7
16906     6
984160    6
258       5
         ..
902664    1
26722     1
942230    1
43131     1
952078    1
Name: ItemNumber, Length: 11249, dtype: int64

In [126]:
#removes duplicate item numbers
ddfproduct = ddfproduct.drop_duplicates(subset = ['ItemNumber'])

In [127]:
#checks that all value counts are now 1
ddfproduct['ItemNumber'].value_counts(dropna = False)

981177    1
40962     1
901130    1
928910    1
17176     1
         ..
986723    1
986962    1
41951     1
77363     1
978102    1
Name: ItemNumber, Length: 11249, dtype: int64

In [128]:
ddfproduct


,ItemNumber,ItemDescription,Pack,BottleVolume(ml)
0,11296,Crown Royal,12,750
1,43125,Bacardi Superior PET,12,750
2,89387,Juarez Gold,12,"1,000"
3,48099,Hennessy VS,24,200
4,36301,Hawkeye Vodka Mini,12,50
...,...,...,...,...
19277,926556,Chicken Cock Kentucky Straight Rye,6,750
23121,968296,El Dorado Rum Cream Liqueur,12,750
25109,917079,Bull Run Distillery 15 year Bourbon,12,750
28183,944015,Rhum JM XO,6,750


In [129]:
#identifies row with letter in item number
ddfproduct[ddfproduct['ItemNumber'] == 'x904631']

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)
9982,x904631,Tanqueray Gin Mini - Use 904631 Code,12,500


In [130]:
#removes letter from item number
ddfproduct = ddfproduct.replace('x904631', '904631')


In [131]:
#checks to see if all value counts are still 1
ddfproduct['ItemNumber'].value_counts(dropna = False)


904631    2
981177    1
27197     1
928910    1
17176     1
         ..
68958     1
986723    1
986962    1
41951     1
978102    1
Name: ItemNumber, Length: 11248, dtype: int64

In [134]:
#removes duplicate item numbers
ddfproduct = ddfproduct.drop_duplicates(subset = ['ItemNumber'])


In [132]:
#replace any commas in the bottlevolume column
ddfproduct['BottleVolume(ml)']=ddfproduct['BottleVolume(ml)'].str.replace(',','')

#replace any NANs with 0
ddfproduct['BottleVolume(ml)']=ddfproduct['BottleVolume(ml)'].fillna(0)

In [133]:
ddfproduct

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)
0,11296,Crown Royal,12,750
1,43125,Bacardi Superior PET,12,750
2,89387,Juarez Gold,12,1000
3,48099,Hennessy VS,24,200
4,36301,Hawkeye Vodka Mini,12,50
...,...,...,...,...
19277,926556,Chicken Cock Kentucky Straight Rye,6,750
23121,968296,El Dorado Rum Cream Liqueur,12,750
25109,917079,Bull Run Distillery 15 year Bourbon,12,750
28183,944015,Rhum JM XO,6,750


In [135]:
#convert cols to integer type
ddfproduct.loc[:, 'ItemNumber'] = ddfproduct['ItemNumber'].astype(int)
ddfproduct.loc[:, 'Pack'] = ddfproduct['Pack'].astype(int)
ddfproduct.loc[:, 'BottleVolume(ml)'] = ddfproduct['BottleVolume(ml)'].astype(int)


In [136]:
#add in row for unknown
ddfproduct = ddfproduct.append(pd.Series({'ItemNumber': 99999999999, 'ItemDescription': 'Unknown', 'Pack': 0, 'BottleVolume(ml)': 0}), ignore_index = True)
ddfproduct

,ItemNumber,ItemDescription,Pack,BottleVolume(ml)
0,11296,Crown Royal,12,750
1,43125,Bacardi Superior PET,12,750
2,89387,Juarez Gold,12,1000
3,48099,Hennessy VS,24,200
4,36301,Hawkeye Vodka Mini,12,50
...,...,...,...,...
11244,968296,El Dorado Rum Cream Liqueur,12,750
11245,917079,Bull Run Distillery 15 year Bourbon,12,750
11246,944015,Rhum JM XO,6,750
11247,927274,Two Jays Light Whiskey,12,375


In [138]:
#export df as csv
ddfproduct.to_csv('product.csv')